<a href="https://colab.research.google.com/github/LiibanMo/ML-Algorithms/blob/main/DecisionTrees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [347]:
import numpy as np

In [348]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [349]:
def obtain_unique_vals(data, col):
    return set(row[col] for row in data)

In [350]:
obtain_unique_vals(training_data, 2)

{'Apple', 'Grape', 'Lemon'}

In [351]:
def class_counts(data):
    counts = {}
    for row in data:
        label = row[-1]
        counts[label] = counts.get(label, 0) + 1
    return counts

In [352]:
sum(class_counts(training_data).values())

5

In [353]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def evaluate(self, example):
        val = example[self.column]
        if isinstance(val, (int, float)):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        if isinstance(self.value, (int, float)):
            return f"Question: Is row[{self.column}] >= {self.value}?"
        else:
            return f"Question: Is row[{self.column}] == {self.value}?"

In [354]:
def partition(data, question):
    true_rows, false_rows = [], []
    for row in data:
        if question.evaluate(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [355]:
question = Question(0, 'Red')
true_rows, false_rows = partition(training_data, question)

In [356]:
def gini(rows):
    counts = class_counts(rows)
    gini_idx = 1
    for freq in counts.values():
        prob = freq / sum(counts.values())
        gini_idx -= prob ** 2
    return gini_idx

In [357]:
def info_gain(left, right, current_uncertainty):
    weight = len(left) / (len(right) + len(left))
    return current_uncertainty - weight * gini(left) - (1 - weight) * gini(right)

In [358]:
current_uncertainty = gini(training_data)
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [359]:
def find_best_split(data):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(data)
    num_features = len(data[0]) - 1

    for column in range(num_features):
        values = set(row[column] for row in data)
        for value in values:
            question = Question(column, value)
            true_rows, false_rows = partition(data, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(false_rows, true_rows, current_uncertainty)
            if gain > best_gain:
                best_gain = gain
                best_question = question

    return best_gain, best_question

In [360]:
best_gain, best_question = find_best_split(training_data)
best_question

Question: Is row[0] == Red?

In [361]:
class Leaf:
    def __init__(self, data):
        self.predictions = class_counts(data)

In [362]:
class DecisionNode:
    def __init__(self, question, right_branch, left_branch):
        self.question = question
        self.true_branch = right_branch
        self.false_branch = left_branch

In [363]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self._data = []
        for x_i, y_i in zip(X, y):
            if isinstance(x_i, list):
                self._data.append(x_i + [y_i])
            else:
                self._data.append([x_i] + [y_i])

        def partition(data, question):
            true_rows, false_rows = [], []
            for row in data:
                if question.evaluate(row):
                    true_rows.append(row)
                else:
                    false_rows.append(row)
            return true_rows, false_rows

        def gini(rows):
            counts = class_counts(rows)
            gini_idx = 1
            for freq in counts.values():
                prob = freq / sum(counts.values())
                gini_idx -= prob ** 2
            return gini_idx

        def info_gain(left, right, current_uncertainty):
            weight = len(left) / (len(right) + len(left))
            return current_uncertainty - weight * gini(left) - (1 - weight) * gini(right)

        def find_best_split(data):
            best_gain = 0
            best_question = None
            current_uncertainty = gini(data)
            num_features = len(data[0]) - 1

            for column in range(num_features):
                values = obtain_unique_vals(data, column)
                for value in values:
                    question = Question(column, value)
                    true_rows, false_rows = partition(data, question)
                    if len(true_rows) == 0 or len(false_rows) == 0:
                        continue
                    gain = info_gain(true_rows, false_rows, current_uncertainty)
                    if gain > best_gain:
                        best_gain = gain
                        best_question = question

            return best_gain, best_question

        self.depth = 0
        def build_tree(data):
            gain, question = find_best_split(data)
            if gain == 0 or self.depth == self.max_depth:
                return Leaf(data)

            true_branch, false_branch = partition(data, question)

            self.depth += 1
            right_branch = build_tree(true_branch)
            left_branch = build_tree(false_branch)

            return DecisionNode(question, right_branch, left_branch)

        self.tree = build_tree(self._data)

        return self

    def predict(self, X):
        def classify(x, node):
            if isinstance(node, Leaf):
                return node.predictions
            if node.question.evaluate(x):
                return classify(x, node.true_branch)
            else:
                return classify(x, node.false_branch)

        predictions = []
        if isinstance(X[0], list):
            for x in X:
                predictions.append(classify(x, self.tree))
        else:
            predictions = classify(X, self.tree)
        return predictions

In [368]:
X = []
y = []
for row in training_data:
    X.append(row[:-1])
    y.append(row[-1])

training_data

[['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [373]:
model = DecisionTree(max_depth=100)
fitted_model = model.fit(X, y)
fitted_model.predict(training_data)

[{'Apple': 1},
 {'Apple': 1, 'Lemon': 1},
 {'Grape': 2},
 {'Grape': 2},
 {'Apple': 1, 'Lemon': 1}]